# Distance Analysis

Notebook for move-off and distance analysis for the whole OpenDD dataset.

In [1]:
import sys
import pathlib
from itertools import product, tee
from dataclasses import dataclass
from typing import Tuple, List, Optional
import mathcreate_logger

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import numpy as np
import momepy
from shapely import geometry, ops
from shapely.geometry import LineString
from tqdm.auto import tqdm

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sys.path.append("..")

In [5]:
from src.preprocess import transform_df_to_trajectory_gdf
from src.visualize import random_colors
from src.trajectory import Trajectory
from src.opendd import extract_samples_from_sqlite

In [6]:
mpl.rcParams["figure.facecolor"] = "w"

# Prepare

In [13]:
RAW_DATA_PATH = pathlib.Path("../data/raw")
INTERIM_DATA_PATH = pathlib.Path("../data/interim")

In [14]:
FIGURES_PATH = pathlib.Path("../reports/figures")

In [15]:
ROUNDABOUT_DIRS = list(RAW_DATA_PATH.rglob(r"rdb[1-9]"))

In [31]:
ROUNDABOUTS = [d.name for d in ROUNDABOUT_DIRS]

In [32]:
ROUNDABOUTS

['rdb1', 'rdb2', 'rdb3']

# Extract reference paths

In [16]:
from src.path_extraction import visualize_traffic_lanes, traffic_lanes_to_graph, \
    extract_paths_from_graph, driving_path_overview_plot

In [9]:
reference_path_lookup = dict()

In [17]:
for roundabout_dir in ROUNDABOUT_DIRS:
    
    roundabout_name = roundabout_dir.name
    print(f"Extracting reference paths from '{roundabout_name}'")
    
    shapefiles_trafficlanes_path = roundabout_dir / f"map_{roundabout_name}/shapefiles_trafficlanes"
    
    trafficlanes = gpd.read_file(shapefiles_trafficlanes_path)
    
    G = traffic_lanes_to_graph(trafficlanes)
    
    paths = extract_paths_from_graph(G)
    
    # save in lookup
    reference_path_lookup[roundabout_name] = paths
    
    fig = driving_path_overview_plot(trafficlanes, paths)
    fig.suptitle(roundabout_name, y=1.01, fontsize=15)
    
    # save images
    # roundabout_paths_overview_plot_path = FIGURES_PATH / f"{roundabout_name}_reference_paths.png"
    # fig.savefig(roundabout_paths_overview_plot_path, dpi=200)
    # plt.close(fig)

Extracting reference paths from 'rdb1'
Extracting reference paths from 'rdb2'
Extracting reference paths from 'rdb3'


# Read data

In [36]:
for roundabout_dir in ROUNDABOUT_DIRS:
    
    roundabout_name = roundabout_dir.name
    print(f"Extracting '{roundabout_name}'")
    
    # Extract database
    roundabout_db = list(roundabout_dir.glob("*.sqlite"))[0]    
    df_roundabout = extract_samples_from_sqlite(roundabout_db)
    
    # Save to disk
    output_path_roundabout = INTERIM_DATA_PATH / f"{roundabout_name}.parquet"
    df_roundabout.to_parquet(output_path_roundabout)

Extracting 'rdb1'
Found 153 measurements


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 153/153 [02:08<00:00,  1.19it/s]


Extracting 'rdb2'
Found 56 measurements


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [00:31<00:00,  1.77it/s]


Extracting 'rdb3'
Found 54 measurements


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:24<00:00,  2.24it/s]


# Extract moving-off situations

In [74]:
from src.preprocess import VehicleState, identify_driving_state
from src.path_assignment import find_nearest_path, DrivablePath
from src.reference_path import DiscreteReferencePath
from src.drive_off import analyze_driveoffs_from_path
from src.opendd import VEHICLE_CLASSES
from src.utils import create_logger

In [75]:
__name__

'__main__'

In [76]:
logger = create_logger("extraction")

In [77]:
tqdm.pandas()

In [82]:
def trace_to_frenet(trace: pd.Series, reference_paths: List[DiscreteReferencePath]) -> LineString:
    path_id = trace["path_id"]
    ref_path = reference_paths[path_id]
    return ref_path.linestring_to_frenet(trace["geometry"])

In [83]:
def extract_moving_off_situations(roundabout_samples: pd.DataFrame, paths: List[DrivablePath]) -> pd.DataFrame:
    
    result_list = list()
    
    num_measurements = roundabout_samples["MEASUREMENT"].nunique()
    
    logger.info(f"Found {num_measurements:d} measurements.")
    
    for meas, df_measurement in roundabout_samples.groupby("MEASUREMENT"):
        
        if len(df_measurement) < 1:
            continue
        
        gdf_traces = transform_df_to_trajectory_gdf(df_measurement)

        # keep only motorized vehicles
        gdf_traces = gdf_traces[gdf_traces["CLASS"].isin(VEHICLE_CLASSES)]

        # identify driving state
        gdf_traces["STATE"] = gdf_traces.apply(lambda row: identify_driving_state(row), axis=1)
        
        logger.info(f"Assigning {len(gdf_traces):d} trajectories to path")

        # assign to reference path
        gdf_traces["path_id"] = gdf_traces.progress_apply(lambda row: find_nearest_path(row["geometry"], paths, N=50), axis=1)

        # create discrete reference paths
        reference_paths = [DiscreteReferencePath.from_linestring(dp.as_linestring(), resolution=0.05) \
                           for dp in paths]

        # transform to frenet (frenet_path is a linestring, x=tangential, y=normal)
        gdf_traces["frenet_path"] = gpd.GeoSeries(gdf_traces.apply(lambda row: trace_to_frenet(row, reference_paths), axis=1))

        # extract the first frenet coordinate
        gdf_traces["S"] = gdf_traces["frenet_path"].apply(lambda row: row.xy[0])

        # for each path, find the drive-off situations
        logger.info(f"Retrieving drive-off situations.")
        df_situations = gdf_traces.groupby("path_id").progress_apply(analyze_driveoffs_from_path)
        df_situations = df_situations.reset_index(drop=True)
        df_situations["o1_id"] = df_situations["o1_id"].astype(int)
        df_situations["o2_id"] = df_situations["o2_id"].astype(int)
        df_situations["o2_state"] = df_situations["o2_state"].astype(int)
        
        # record measurement name
        df_situations["measurement"] = meas
        
        result_list.append(df_situations)
        
    df_situations = pd.concat(result_list)
    df_situations["measurement"] = df_situations["measurement"].astype("category")
    
    return df_situations

In [84]:
result_list = list()

for i, rb in enumerate(ROUNDABOUTS):
    
    logger.info(f"Processing {rb}")
    
    # read samples from disk
    samples_path = INTERIM_DATA_PATH / f"{rb}.parquet"
    df_roundabout = pd.read_parquet(samples_path)
    
    logger.info(f"Data loaded!")
    
    paths = reference_path_lookup[rb]
    logger.info(f"{len(paths)} paths loaded!")
    
    df_situations = extract_moving_off_situations(df_roundabout, paths)
    df_situations["roundabout"] = rb
    
    result_list.append(df_situations)
    
df_result = pd.concat(result_list)
df_result["roundabout"] = df_result["roundabout"].astype("category")

2022-02-17 19:11:25,639 - extraction - INFO - Processing rdb1
2022-02-17 19:11:25,639 - extraction - INFO - Processing rdb1
2022-02-17 19:11:25,639 - extraction - INFO - Processing rdb1
2022-02-17 19:11:27,156 - extraction - INFO - Data loaded!
2022-02-17 19:11:27,156 - extraction - INFO - Data loaded!
2022-02-17 19:11:27,156 - extraction - INFO - Data loaded!
2022-02-17 19:11:27,297 - extraction - INFO - Found 153 measurements.
2022-02-17 19:11:27,297 - extraction - INFO - Found 153 measurements.
2022-02-17 19:11:27,297 - extraction - INFO - Found 153 measurements.
2022-02-17 19:11:29,055 - extraction - INFO - Assigning 332 trajectories to path
2022-02-17 19:11:29,055 - extraction - INFO - Assigning 332 trajectories to path
2022-02-17 19:11:29,055 - extraction - INFO - Assigning 332 trajectories to path


  0%|          | 0/332 [00:00<?, ?it/s]

2022-02-17 19:12:21,340 - extraction - INFO - Retrieving drive-off situations from 30 paths.
2022-02-17 19:12:21,340 - extraction - INFO - Retrieving drive-off situations from 30 paths.
2022-02-17 19:12:21,340 - extraction - INFO - Retrieving drive-off situations from 30 paths.


  0%|          | 0/18 [00:00<?, ?it/s]

2022-02-17 19:12:22,903 - extraction - INFO - Assigning 313 trajectories to path
2022-02-17 19:12:22,903 - extraction - INFO - Assigning 313 trajectories to path
2022-02-17 19:12:22,903 - extraction - INFO - Assigning 313 trajectories to path


...

2022-02-17 19:16:21,743 - extraction - INFO - Retrieving drive-off situations from 30 paths.
2022-02-17 19:16:21,743 - extraction - INFO - Retrieving drive-off situations from 30 paths.
2022-02-17 19:16:21,743 - extraction - INFO - Retrieving drive-off situations from 30 paths.


  0%|          | 0/9 [00:00<?, ?it/s]

2022-02-17 19:16:22,374 - extraction - INFO - Assigning 239 trajectories to path
2022-02-17 19:16:22,374 - extraction - INFO - Assigning 239 trajectories to path
2022-02-17 19:16:22,374 - extraction - INFO - Assigning 239 trajectories to path


  0%|          | 0/239 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [81]:
df_result.head()

,o1_id,o2_id,t,distance,o2_state,o2_timedelta_drive_off,o2_velocity,measurement,roundabout
0,14,17,8.904500,7.20,1,NaN,0.60,rdb1_1,rdb1
1,9,3,2.333333,7.55,1,NaN,0.22,rdb1_1,rdb1
2,3,4,4.233333,5.75,0,0.1,0.14,rdb1_1,rdb1
3,4,11,4.333333,10.25,1,NaN,0.44,rdb1_1,rdb1
4,4,11,5.133333,9.90,1,NaN,0.42,rdb1_1,rdb1
